In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import stats
from sklearn.preprocessing import LabelEncoder, StandardScaler,MultiLabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from lightgbm import LGBMClassifier

In [ ]:
column = ['Class', 'Message']
#df = pd.read_csv('SMSSpamCollection')
df = pd.read_csv('../input/spam.csv',delimiter=',',encoding='latin-1')
df = df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'])
df.rename(columns={'v1':'Class', 'v2':'Message'}, inplace= True)
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df[df['Class'] == 'spam'].describe()

In [ ]:
df[df['Class'] == 'ham'].describe()

In [ ]:
df.groupby('Class').describe().T

In [ ]:
df.head()

In [ ]:
df['Mes_len'] = df['Message'].apply(len)

In [ ]:
df.head()

In [ ]:
df.groupby('Class').describe().T

In [ ]:
df['Mes_len'].plot(kind='hist',bins=50)
plt.show()

In [ ]:
sns.barplot(df['Mes_len'],df['Class'])
plt.show()

In [ ]:
plt.bar(df['Class'], df['Mes_len'])
#plt.xlabel()
#plt.legend()
label = ['spam', 'ham']
plt.show()

In [ ]:
df.groupby('Class')['Mes_len'].max()

In [ ]:
# lets check the message with the longest length for both classes
print(df.loc[df['Mes_len']==910,'Message'][1084])


In [ ]:
print(df.loc[df['Mes_len']==224,'Message'][1020])

### This means that the classification seems to be apt enough to be taken as correct

In [ ]:
df.hist(column='Mes_len', by='Class', bins=50, figsize=(10,4))
plt.show()

It means usually shorter messages are hams and longer messages are spams. Hence, classifiers such as Naive Bayes might turnout to be a success over here

In [ ]:
#Removing Stopwords & Punctuations and applying Lemmatising/Stemming
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk import stem
import string

def token(example_sent):
    
    stemmer = stem.SnowballStemmer('english')
    
    stop_words = set(stopwords.words('english') + list(string.punctuation))

    word_tokens = word_tokenize(example_sent) 

    filtered_sentence = [w for w in word_tokens if w.lower() not in stop_words] 
    
    msg = [stemmer.stem(word) for word in filtered_sentence]
    return(msg)


In [ ]:
df['Message'].apply(token)

In [ ]:
df.head()

In [ ]:
bag_of_words = CountVectorizer(token).fit(df['Message'])

In [ ]:
message_bag_of_words = bag_of_words.transform(df['Message'])

In [ ]:
message_bag_of_words.nnz


In [ ]:
tf_idf_trans = TfidfTransformer().fit(message_bag_of_words)

In [ ]:
messages_tfidf_trans = tf_idf_trans.transform(message_bag_of_words)

In [ ]:
spam_detector_model = MultinomialNB().fit(messages_tfidf_trans,df['Message'])

In [ ]:
msg_Train,msg_Test,Classification_train, Classification_test=train_test_split(df['Message'],df['Class'],test_size=0.3, random_state=123)



In [ ]:
pipeline=Pipeline([
    ('bow',CountVectorizer(token)),
    ('Tfidf',TfidfTransformer()),
    ('NBMultinoial',MultinomialNB())
])
#Using Pipleline to perform differnt stepe in one go

In [ ]:
pipeline.fit(msg_Train,Classification_train)

In [ ]:
prediction=pipeline.predict(msg_Test)

In [ ]:
print(classification_report(Classification_test,prediction))

In [ ]:
#Using Random forest for the prediction
pipeline1=Pipeline([
    ('bow',CountVectorizer(token)),
    ('Tfidf',TfidfTransformer()),
    ('Randomforest',RandomForestClassifier(random_state=123))
])

In [ ]:
pipeline1.fit(msg_Train,Classification_train)

In [ ]:
prediction1=pipeline1.predict(msg_Test)

In [ ]:
print(classification_report(Classification_test,prediction1))

In [ ]:
#Using Light GBM for the prediction
pipeline2=Pipeline([
    ('bow',CountVectorizer(token)),
    ('Tfidf',TfidfTransformer()),
    ('LightGBM',LGBMClassifier(random_state=123))
])

In [ ]:
pipeline2.fit(msg_Train,Classification_train)

In [ ]:
prediction2=pipeline2.predict(msg_Test)

In [ ]:
print(classification_report(Classification_test,prediction2))